In [ ]:
!pip install geopandas
!pip install geopy
!pip install folium

     |████████████████████████████████| 972kB 3.8MB/s 
     |████████████████████████████████| 6.5MB 51.9MB/s 
     |████████████████████████████████| 14.8MB 289kB/s 


In [ ]:
import folium 
import csv
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from folium import plugins
from folium.plugins import HeatMap

In [ ]:
df = pd.read_csv("gurugram_wards.csv", encoding= 'unicode_escape')

print(df)

     Ward Number                          Ward
0     Ward No. 1       New Palam Vihar Phase 1
1     Ward No. 1       New Palam Vihar Phase 2
2     Ward No. 1       New Palam Vihar Phase 3
3     Ward No. 1                  Nihal Colony
4     Ward No. 1              Pawala Khusrupur
..           ...                           ...
345  Ward No. 35              DLF City Phase 3
346  Ward No. 35                      Nathupur
347  Ward No. 35              Nathupur Village
348  Ward No. 35         National Media Center
349  Ward No. 35  Pahari Area Nathupur Village

[350 rows x 2 columns]


In [ ]:
list(df)

['Ward Number', 'Ward']

In [ ]:
df['Country'] = 'India'
df.head()

,Ward Number,Ward,Country
0,Ward No. 1,New Palam Vihar Phase 1,India
1,Ward No. 1,New Palam Vihar Phase 2,India
2,Ward No. 1,New Palam Vihar Phase 3,India
3,Ward No. 1,Nihal Colony,India
4,Ward No. 1,Pawala Khusrupur,India


In [ ]:
df['State'] = 'Gurugram'
df.head()

,Ward Number,Ward,Country,State
0,Ward No. 1,New Palam Vihar Phase 1,India,Gurugram
1,Ward No. 1,New Palam Vihar Phase 2,India,Gurugram
2,Ward No. 1,New Palam Vihar Phase 3,India,Gurugram
3,Ward No. 1,Nihal Colony,India,Gurugram
4,Ward No. 1,Pawala Khusrupur,India,Gurugram


In [ ]:
df["location"] = df['Ward'] +", " + df['State'] +", " + df['Country']
df.head()

,Ward Number,Ward,Country,State,location
0,Ward No. 1,New Palam Vihar Phase 1,India,Gurugram,"New Palam Vihar Phase 1, Gurugram, India"
1,Ward No. 1,New Palam Vihar Phase 2,India,Gurugram,"New Palam Vihar Phase 2, Gurugram, India"
2,Ward No. 1,New Palam Vihar Phase 3,India,Gurugram,"New Palam Vihar Phase 3, Gurugram, India"
3,Ward No. 1,Nihal Colony,India,Gurugram,"Nihal Colony, Gurugram, India"
4,Ward No. 1,Pawala Khusrupur,India,Gurugram,"Pawala Khusrupur, Gurugram, India"


In [ ]:
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim 
import numpy as np
   
# declare an empty list to store 
# latitude and longitude of values  
# of city column 
longitude = [] 
latitude = []       
  
# function to find the coordinate 
# of a given city  
def findGeocode(location): 
       
    # try and catch is used to overcome 
    # the exception thrown by geolocator 
    # using geocodertimedout   
    try: 
          
        # Specify the user_agent as your 
        # app name it should not be none 
        geolocator = Nominatim(user_agent="myGeocoder")
        
        return geolocator.geocode(location) 
      
    except GeocoderTimedOut: 
          
        return findGeocode(location) 
# each value from city column 
# will be fetched and sent to 
# function find_geocode    
for i in (df["location"]): 
      
    if findGeocode(i) != None: 
           
        loc = findGeocode(i) 
          
        # coordinates returned from  
        # function is stored into 
        # two separate list 
        latitude.append(loc.latitude) 
        longitude.append(loc.longitude) 
       
    # if coordinate for a city not 
    # found, insert "NaN" indicating  
    # missing value  
    else: 
        latitude.append(np.nan) 
        longitude.append(np.nan) 

In [ ]:
df["Longitude"] = longitude 
df["Latitude"] = latitude 

df.head()

,Ward Number,Ward,Country,State,location,Longitude,Latitude
0,Ward No. 1,New Palam Vihar Phase 1,India,Gurugram,"New Palam Vihar Phase 1, Gurugram, India",77.042956,28.517956
1,Ward No. 1,New Palam Vihar Phase 2,India,Gurugram,"New Palam Vihar Phase 2, Gurugram, India",77.042956,28.517956
2,Ward No. 1,New Palam Vihar Phase 3,India,Gurugram,"New Palam Vihar Phase 3, Gurugram, India",77.020301,28.498335
3,Ward No. 1,Nihal Colony,India,Gurugram,"Nihal Colony, Gurugram, India",NaN,NaN
4,Ward No. 1,Pawala Khusrupur,India,Gurugram,"Pawala Khusrupur, Gurugram, India",NaN,NaN


In [ ]:
df.to_csv("gurugram.csv")

In [ ]:
map_hooray = folium.Map(location=[28.510001, 77.043500], zoom_start = 13) 

# Ensure you're handing it floats
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
heat_df = df[['Latitude', 'Longitude']]
heat_df = heat_df.dropna(axis=0, subset=['Latitude','Longitude'])

# List comprehension to make out list of lists
heat_data = [[row['Latitude'],row['Longitude']] for index, row in heat_df.iterrows()]

# Plot it on the map
HeatMap(heat_data).add_to(map_hooray)

# Display the map
map_hooray

In [ ]:
df['Latitude']= df['Latitude'].fillna(0)
df['Longitude']= df['Longitude'].fillna(0)
df.head()

,Ward Number,Ward,Country,State,location,Longitude,Latitude
0,Ward No. 1,New Palam Vihar Phase 1,India,Gurugram,"New Palam Vihar Phase 1, Gurugram, India",77.042956,28.517956
1,Ward No. 1,New Palam Vihar Phase 2,India,Gurugram,"New Palam Vihar Phase 2, Gurugram, India",77.042956,28.517956
2,Ward No. 1,New Palam Vihar Phase 3,India,Gurugram,"New Palam Vihar Phase 3, Gurugram, India",77.020301,28.498335
3,Ward No. 1,Nihal Colony,India,Gurugram,"Nihal Colony, Gurugram, India",0.000000,0.000000
4,Ward No. 1,Pawala Khusrupur,India,Gurugram,"Pawala Khusrupur, Gurugram, India",0.000000,0.000000


In [ ]:
latitude = 28.510001
longitude = 77.043500
faridabad_map = folium.Map(location=[latitude, longitude], zoom_start=13)

In [ ]:
for lat, lon, ward in zip(df['Latitude'], df['Longitude'], df['Ward Number']):
    folium.CircleMarker(
        [lat, lon],
        popup = ('Ward: ' + str(ward).capitalize()
                ),
        color='red',
        key_on = ward,
        threshold_scale=[0,1,2,3],
        fill=True,
        fill_opacity=0.7
        ).add_to(faridabad_map)
faridabad_map